In [4]:
import json
import glob
import h_gen
import matplotlib.pyplot as plt
from tqdm import tqdm_notebook as tq
import matplotlib.patches as patches
import numpy as np
from PIL import Image
import utils
import importlib
import meta
metadata = utils.load("./variables/metadata.p")
from keras.optimizers import *
from keras.models import Sequential
from keras.layers import *
from keras import backend as K
from keras.preprocessing.image import ImageDataGenerator
from keras import callbacks
import os
from keras.models import Model
from keras.applications.vgg16 import VGG16
from keras.applications.vgg19 import VGG19
remote = callbacks.RemoteMonitor(root='http://localhost:9000')
from keras.layers.normalization import *
from keras.models import load_model
from shutil import copyfile
importlib.reload(utils)
importlib.reload(meta)

<module 'meta' from 'C:\\Users\\yolo\\Desktop\\fish\\meta.py'>

First, we need to create a dataset. 

In [19]:
def split_train_test(nb_images, train_dir):
    temp = train_dir.find("train")
    test_dir = train_dir[:temp] + "test" + train_dir[temp+5:]
    
    if not os.path.exists(test_dir):
        os.makedirs(test_dir)
        
    classes = os.listdir(train_dir)
        
    train_dir = [train_dir + x + "/" for x in classes]
    test_dir = [test_dir + x + "/" for x in classes]
    
    for folder in test_dir:
        if not os.path.exists(folder):
            os.makedirs(folder)
    
    for train_class, test_class in zip(train_dir, test_dir):
        files = os.listdir(train_class)[:nb_images]
        for file in files:
            copyfile(train_class + file,test_class + file)
            os.remove(train_class + file)

In [20]:
split_train_test(30,"cropped_train/")

In [5]:
train_datagen = ImageDataGenerator(
    rotation_range=180,
    width_shift_range=0.01,
    height_shift_range=0.01,
    shear_range=0.0005,
    horizontal_flip=True,
    vertical_flip=True)
test_datagen = ImageDataGenerator()

In [6]:
target_size=(224,224)
train_generator = train_datagen.flow_from_directory('cropped_gen', batch_size=32, 
                                                    target_size=target_size)
validation_generator = test_datagen.flow_from_directory('cropped_test', batch_size=32, target_size=target_size)

Found 15997 images belonging to 8 classes.
Found 240 images belonging to 8 classes.


In [7]:
base = VGG16(include_top=False)
img_input = Input(shape=(3,None,None))
x = base(img_input)
x = BatchNormalization(axis=1)(x)
x =Convolution2D(64, 3,3, activation="relu", border_mode="same")(x)
x = BatchNormalization(axis=1)(x)
x =Convolution2D(64, 3,3, activation="relu", border_mode="same")(x)
x = BatchNormalization(axis=1)(x)
x =Convolution2D(32, 3,3, activation="relu", border_mode="same")(x)
x = BatchNormalization(axis=1)(x)
x =Convolution2D(8, 3,3, border_mode="same")(x)
x = GlobalAveragePooling2D()(x)
x = Activation("softmax")(x)
model = Model(img_input, x)
for layer in base.layers:
    layer.trainable=False

base.trainable=False

In [8]:
model.summary()

____________________________________________________________________________________________________
Layer (type)                     Output Shape          Param #     Connected to                     
input_2 (InputLayer)             (None, 3, None, None) 0                                            
____________________________________________________________________________________________________
vgg16 (Model)                    (None, 512, None, Non 14714688    input_2[0][0]                    
____________________________________________________________________________________________________
batchnormalization_1 (BatchNorma (None, 512, None, Non 2048        vgg16[1][0]                      
____________________________________________________________________________________________________
convolution2d_1 (Convolution2D)  (None, 64, None, None 294976      batchnormalization_1[0][0]       
___________________________________________________________________________________________

In [15]:
#opt = SGD(lr=0.01, decay=0.01)
opt = Adam(lr=0.001, beta_1=0.9, beta_2=0.999, epsilon=1e-08, decay=0.01)
model.compile(optimizer=opt, loss='categorical_crossentropy', metrics=['accuracy'])

In [10]:
# train the model on the new data for a few epochs
model.fit_generator(train_generator,
        samples_per_epoch=512,
        nb_epoch=100,
        validation_data=validation_generator,
        verbose=0,
        nb_val_samples=240, callbacks=[remote])

DEBUG: nvcc STDOUT mod.cu
   Creating library C:/Users/yolo/AppData/Local/Theano/compiledir_Windows-10-10.0.14393-SP0-Intel64_Family_6_Model_60_Stepping_3_GenuineIntel-3.5.2-64/tmpe02t5j09/m9a6bd0eb5ed5c92e91261282fc495cb4.lib and object C:/Users/yolo/AppData/Local/Theano/compiledir_Windows-10-10.0.14393-SP0-Intel64_Family_6_Model_60_Stepping_3_GenuineIntel-3.5.2-64/tmpe02t5j09/m9a6bd0eb5ed5c92e91261282fc495cb4.exp

DEBUG: nvcc STDOUT mod.cu
   Creating library C:/Users/yolo/AppData/Local/Theano/compiledir_Windows-10-10.0.14393-SP0-Intel64_Family_6_Model_60_Stepping_3_GenuineIntel-3.5.2-64/tmpnt2rpg86/m848dd898e26d545ff6290e3aa98de3d5.lib and object C:/Users/yolo/AppData/Local/Theano/compiledir_Windows-10-10.0.14393-SP0-Intel64_Family_6_Model_60_Stepping_3_GenuineIntel-3.5.2-64/tmpnt2rpg86/m848dd898e26d545ff6290e3aa98de3d5.exp

DEBUG: nvcc STDOUT mod.cu
   Creating library C:/Users/yolo/AppData/Local/Theano/compiledir_Windows-10-10.0.14393-SP0-Intel64_Family_6_Model_60_Stepping_3_Genui

KeyboardInterrupt: 

In [11]:
for layer in base.layers:
    layer.trainable=True

base.trainable=True

In [17]:
model.save("yoloswagdu113.h5")

In [18]:
model2 = load_model("yoloswagdu113.h5")

In [6]:
def create_new_pictures(train_folder, class_folder, generator,total_images, 
                        batch_size = 32, target_size = (224,224), color_mode = "rgb"):
    #count = len(os.listdir(train_folder + "/" + class_folder))
    count = 0  
    gen_folder = train_folder + "_gen/"
        
    if not os.path.exists(gen_folder):
        os.makedirs(gen_folder)
        
    if not os.path.exists(gen_folder + class_folder):
        os.makedirs(gen_folder + class_folder)
        
    
    progbar = tq(total = total_images, leave=False)
    progbar.update(count)
    
    
    for X in generator.flow_from_directory(train_folder, target_size=target_size,
                                             classes=[class_folder], 
                                             save_to_dir=train_folder + "_gen/" + class_folder, 
                                             batch_size=batch_size, color_mode=color_mode):
        count +=batch_size
        progbar.update(batch_size)
        if count >= total_images:
            break
    progbar.close()
    

In [9]:
def equilibrate_dataset(folder, nb_images_per_class, generator):
    c = 0
    while c < nb_images_per_class:
        for string in os.listdir(folder):
            create_new_pictures(folder, string, generator, 2000, 1, (224,224))
        c += 2000

In [10]:
equilibrate_dataset("cropped", 2000, test_datagen)

Found 2500 images belonging to 1 classes.
Found 277 images belonging to 1 classes.
Found 94 images belonging to 1 classes.
Found 71 images belonging to 1 classes.
Found 115184 images belonging to 1 classes.
Found 359 images belonging to 1 classes.
Found 166 images belonging to 1 classes.
Found 793 images belonging to 1 classes.
